# The Recording Container: Cedalion's main data structure and a guide to indexing  

This example notebook introduces the main data classes used by cedalion, and provides examples of how to access and index them.

## Overview

**The class `cedalion.dataclasses.Recording` is Cedalion's main data container that can be used to carry related data objects through the program.** It can store time series, masks, auxiliary timeseries, probe, headmodel and stimulus information as well as meta data about the recording.
It has the following properties:

- It resembles the [NIRS group in the snirf specification](https://github.com/fNIRS/snirf/blob/v1.1/snirf_specification.md#nirsi), which provides storage for much of the data stored in a `Recording` (e.g. time series map to [data elements](https://github.com/fNIRS/snirf/blob/v1.1/snirf_specification.md#nirsidataj), [probe](https://github.com/fNIRS/snirf/blob/v1.1/snirf_specification.md#nirsiprobe), [stimulus](https://github.com/fNIRS/snirf/blob/v1.1/snirf_specification.md#nirsistimj) and [meta data](https://github.com/fNIRS/snirf/blob/v1.1/snirf_specification.md#nirsimetadatatags) are stored per NIRS element, etc). Consequently, the methods `cedalion.io.read_snirf` and `cedalion.io.write_snirf` methods operate on lists of recordings.
- different time series and masks are stored in ordered dictionaries
  - the user differentiates time series by name
  - there is a set of canonical names used by `read_snirf` to assign names to time series
    ```
    CANONICAL_NAMES = {
          "unprocessed raw": "amp",
          "processed raw": "amp",
          "processed dOD": "od",
          "processed concentrations": "conc",
          "processed central moments": "moments",
          "processed blood flow index": "bfi",
          "processed HRF dOD": "hrf_od",
          "processed HRF central moments": "hrf_moments",
          "processed HRF concentrations": "hrf_conc",
          "processed HRF blood flow index": "hrf_bfi",
          "processed absorption coefficient": "mua",
          "processed scattering coefficient": "musp",
    }
    ```
- time series are stored in the dictionaries in the order that they were added
- convenient access to the last changed time series + canonical names -> consecutive transformations of time series without the need to specify time series by name -> workflows
- `rec[key]` is a shortcut for `rec.timeseries[key]` 
- not all information stored in a `Recording` can be stored in snirf files, e.g. for masks, the headmodel and auxiliar objects there is no provision in the snirf specification. We will probably use sidecard files or sidecar hdf groups to store these.

![Recording Container](/img/recording/rec_container_overview.png)

## Exploring the recording container fields with some example data

In [69]:
import os
import cedalion
import cedalion.io
import cedalion.datasets
import cedalion.xrutils as xrutils
import xarray as xr

# Loading an example dataset will create a recording container. 
# Alternatively you can load your ow snirf file using cedalion.io.snirf.read_snirf(PATH_TO_FILE)
rec = cedalion.datasets.get_fingertapping()
display(rec)

<Recording |  timeseries: ['amp'],  masks: [],  stim: ['1.0', '15.0', '2.0', '3.0'],  aux_ts: [],  aux_obj: []>

### The timeseries field
we loaded raw amplitude data and can now access it:

In [70]:
display(rec["amp"])

Magnitude,[[[0.0913686 0.0909875 0.0910225 ... 0.0941083 0.0940129 0.0944882] [0.1856806 0.186377 0.1836514 ... 0.1856486 0.1850836 0.1842172]] [[0.227516 0.2297024 0.2261366 ... 0.2264519 0.2271665 0.226713] [0.6354927 0.637668 0.6298023 ... 0.6072068 0.6087293 0.6091066]] [[0.1064704 0.1066212 0.1053444 ... 0.121114 0.1205022 0.1205441] [0.2755033 0.2761615 0.2727006 ... 0.2911952 0.2900544 0.2909847]] ... [[0.2027881 0.1996586 0.2004866 ... 0.2318743 0.2311941 0.2330808] [0.4666358 0.4554404 0.4561614 ... 0.4809749 0.4812827 0.4862896]] [[0.4885007 0.4802285 0.4818338 ... 0.6109142 0.6108118 0.613845] [0.8457658 0.825988 0.8259648 ... 0.975894 0.9756599 0.9826459]] [[0.6304559 0.6284427 0.6287045 ... 0.6810626 0.6809573 0.6818709] [1.2285622 1.2205907 1.2190002 ... 1.2729124 1.2727222 1.2755645]]]
Units,volt


Since we are interested not only in raw "amp"litude data, we convert this data to concentration using the modified beer-lambert law and save it under **"conc"** in the recording container

In [71]:
import cedalion.nirs

# define DPFs and convert to HbO/HbR using the beer lambert law law
dpf = xr.DataArray(
        [6, 6],
        dims="wavelength",
        coords={"wavelength": rec["amp"].wavelength},
    )
rec["conc"] = cedalion.nirs.beer_lambert(rec["amp"], rec.geo3d, dpf)

display(rec["conc"])

Magnitude,[[[0.13358239209978298 0.003830283928282594 0.3485663033081771 ... 0.44265804481800175 0.5026875842869235 0.6632956216585129] [-0.7839218209250455 -0.7640062849016894 -0.6372321468890523 ... 0.23076741243710222 0.20512673319586014 0.16981311715887415] [0.11212379127914074 0.0731324774375613 0.2338866759571778 ... 0.15960301220515644 0.19659631898706503 0.1288759014200041] ... [-0.08996858531937914 0.29760277238751043 0.3035425193755599 ... 0.570968554065664 0.5273568764191446 0.37714069302428926] [0.9042366397918423 1.340078204934004 1.381330424752029 ... -0.27197257740205283 -0.26748647744434617 -0.40128365125815096] [-0.18579734906426823 0.42721870170643006 0.6069404693046488 ... -0.30557630132353186 -0.29587728904779226 -0.4927704425869436]] [[0.4302890462699156 0.5291122886883187 0.39408017947069485 ... -0.03812950019946504 -0.048780953537845895 -0.16954954368367178] [0.19718416808530376 0.07475108502790183 0.21473067649191507 ... -0.13048136047941516 -0.15862865728380518 -0.12125825687699171] [1.1069744947570999 1.1055519748193772 1.1824229590870727 ... -0.38395393707435815 -0.34006765489373475 -0.31841423445257] ... [1.1310069656643422 1.1631091585413527 1.1132916692449673 ... -0.6597509742064699 -0.6094788888477872 -0.6460548286777947] [1.8250852563797992 1.8988537984724563 1.8366005072863651 ... -0.855162631152548 -0.8545173888400667 -0.8731183537487607] [3.671881628238195 3.6383236051532477 3.5444730339882793 ... -1.0728951396816504 -1.0669728023777476 -1.0756417750550116]]]
Units,micromolar


### The geo3d field
we have already used channel distances from this field to calculate the concentrations using the beer-lambert law above. 
The geo3d and geo2d fields are DataArrays of geometric points, whose "magnitude" is the 3d coordinate in 3D / 2D space. They also have two coordinates: a "label", such as "S1" for Source 1, and a "type" of PointType.Source

In [72]:
display(rec.geo3d)

Magnitude,[[-0.041613204679326624 0.026799775287857947 0.1299043936308115] [-0.06476686499872276 0.05814256998996063 0.0908425773727145] [-0.07120554551675068 -0.012874272652217859 0.10787860947691345] [-0.0859043654400404 0.018971698468891116 0.06509762433137256] [0.03694171596700852 0.02748380530252158 0.13022129709104263] [0.06065133742692848 0.05882414589197514 0.09117717995727878] [0.06712771392323756 -0.012199231886346213 0.1085725493643022] [0.08188685574250908 0.020427932162352107 0.06571325110115192] [-0.037619588707178915 0.06322851630256272 0.11572802770110814] [-0.04134445059646741 -0.011779611291995052 0.13495002938154654] [-0.07242424650162711 0.02347293206381116 0.10322218957482163] [-0.07912592748234686 0.05140929117919257 0.057370046083468226] [0.03352717285472944 0.06359968341212022 0.11583881331702946] [0.03686639505686032 -0.011397164907962862 0.13536724076864515] [0.06791592703520163 0.02468254467119271 0.10366605207860985] [0.075310088095807 0.05226884499005337 0.05787698428594235] [-0.03773895423262196 0.034082658086024245 0.1294919790818403] [-0.061454307897075164 0.06443800208211416 0.09061004226260877] [-0.07282878975853647 -0.00527870527992114 0.10743054838539287] [-0.08439610638498087 0.02706123378098264 0.06559510739262155] [0.040013338219712126 0.020439745814301982 0.13063767506528579] [0.06428020193514211 0.05162125732852231 0.09133632943784001] [0.06521393141744246 -0.019260368037897515 0.10880928230870081] [0.08272091030272573 0.012990608473329186 0.06658402323335233] [-0.0824899918305801 3.5272652784690273e-09 -8.985265795291575e-10] [6.534060185275914e-12 0.11404663614484922 -8.956669156345853e-09] [0.08248999697928468 3.893090638057428e-09 4.766247813092761e-10] [-0.04018770669918394 0.044642295725887106 0.12357659157001165] [-0.04174110787598461 0.007685839199884737 0.13437743644514044] [-0.05885642692737942 0.026136335712672674 0.11745327806321545] [0.03851939726517181 0.03078283979366837 0.1281798987708399] [-0.052808259274512416 0.06188780045911764 0.10403189889709587] [-0.06922421433143165 0.04108974212744533 0.0972095427514579] [-0.07351067792317667 0.05556043944468993 0.07592438053279707] [0.031418413207589764 0.05609701242391968 0.12112072355182572] [-0.05798909959739463 -0.013176608236818512 0.12293491786782985] [-0.07419549041066105 0.00549319711704184 0.10669158774707252] [0.023828589288117673 0.003996293896988853 0.14081960191362605] [-0.08184195746940305 0.022098763709400317 0.08422485645365857] [-0.08309090979477994 0.03520830182778492 0.0610323268526085] [0.026425335781627597 0.043375220063173334 0.12939415639453739] [0.035781770725957916 0.04568049698512951 0.12354633505671649] [0.037556277679295876 0.008001190852073497 0.134555406719549] [0.054080633753515184 0.026705026363902817 0.11818464121484251] [0.06801076312164317 0.02284818084680601 0.10445872040876931] [0.0481712114118228 0.062034828924799924 0.10518292837437133] [0.06461419274656584 0.042329086589563227 0.0977770136200937] [0.06967032405092426 0.05604290375898291 0.07590918892077606] [0.07526945345643418 0.037578919664336574 0.08007431341875021] [0.05288202968424921 -0.012139614237985343 0.12430811820847815] [0.06938013615060833 0.0069016618704050535 0.10806432085401242] [0.07932239267332403 -0.00044800545870984573 0.08903670686337778] [0.07756024143390347 0.022515587935674892 0.08578432016555051] [0.07912303758068968 0.036496511669642975 0.06257633565753129] [0.08225821050778642 0.01751423306824721 0.06619028345490524]]
Units,meter


### The stim field
contains labels for any experimental stimuli that were logged during the recording. Turns out each condition in the experiment was 5 seconds long.

In [73]:
display(rec.stim)

,onset,duration,value,trial_type
0,61.824,5.0,1.0,1.0
1,87.296,5.0,1.0,1.0
2,181.504,5.0,1.0,1.0
3,275.712,5.0,1.0,1.0
4,544.640,5.0,1.0,1.0
...,...,...,...,...
87,2234.112,5.0,1.0,3.0
88,2334.848,5.0,1.0,3.0
89,2362.240,5.0,1.0,3.0
90,2677.504,5.0,1.0,3.0


We can see that the trial_type was encoded numerically, which can be hard to read. If we know the experiment we can rename the stimuli using the "rename_events" function

In [74]:
rec.stim.cd.rename_events(
        {"1.0": "control", "2.0": "Tapping/Left", "3.0": "Tapping/Right"}
    )

display(rec.stim)

,onset,duration,value,trial_type
0,61.824,5.0,1.0,control
1,87.296,5.0,1.0,control
2,181.504,5.0,1.0,control
3,275.712,5.0,1.0,control
4,544.640,5.0,1.0,control
...,...,...,...,...
87,2234.112,5.0,1.0,Tapping/Right
88,2334.848,5.0,1.0,Tapping/Right
89,2362.240,5.0,1.0,Tapping/Right
90,2677.504,5.0,1.0,Tapping/Right


### The masks field
Lastly, we create a **mask** based on an SNR threshold. A mask is a Boolean DataArray that flags each point across all coordinates as either "true" or "false", according to the metric applied. Here we use an SNR of 3 to flag all channels in the raw "amp" timeseries as "False" if their SNR is below the threshold. Since SNR is calculated across the whole time, the time dimension gets dropped. Applying this mask later on to a DataArray time series works implitly thanks to the unambiguous xarray coordinates in the mask and timeseries (here for instance the channel name). 

In [75]:
import cedalion.sigproc.quality as quality
# SNR thresholding using the "snr" function of the quality subpackage using an SNR of 3
_, rec.masks["snr_mask"] = quality.snr(rec["amp"], 3)

display(rec.masks["snr_mask"])

<xarray.DataArray (channel: 28, wavelength: 2)> Size: 56B
array([[ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True]])
Coordinates:
  * channel     (channel) object 224B 'S1D1' 'S1D2' 'S1D3' ... 'S8D8' 'S8D16'
    source      (channel) object 224B 'S1' 'S1' 'S1' 'S1' ... 'S8' 'S8' 'S8'
    detector    (channel) object 224B 'D1' 'D2' 'D3' 'D9' ... 'D7' 'D8' 'D16'
  * wavelength  (wavelength) float64 16B 760.0 850.0

### The headmodel / aux_obj field
The recording container does not yet contain a mask or head model. We load an ICBM152 atlas and create the **headmodel**

In [76]:
import cedalion.imagereco.forward_model as fw

# load segmentation data from the icbm152 atlas
SEG_DATADIR_ic152, mask_files_ic152, landmarks_file_ic152 = cedalion.datasets.get_icbm152_segmentation()

# create forward model class for icbm152 atlas
rec.head_icbm152 = fw.TwoSurfaceHeadModel.from_surfaces(
    segmentation_dir=SEG_DATADIR_ic152,
    mask_files = mask_files_ic152,
    brain_surface_file= os.path.join(SEG_DATADIR_ic152, "mask_brain.obj"),
    landmarks_ras_file=landmarks_file_ic152,
    brain_face_count=None,
    scalp_face_count=None
)

display(rec.head_icbm152)

TwoSurfaceHeadModel(segmentation_masks=<xarray.DataArray (segmentation_type: 5, i: 193, j: 239, k: 263)> Size: 61MB
array([[[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
...
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ...

## xarray  DataArray Indexing and Selecting Data
xarray DataArrays in Cedalion can be indexed "as usual". For a complete documentation visit the [xarray documentation page](https://docs.xarray.dev/en/latest/user-guide/indexing.html). A brief visual overview: 


![DataArray Indexing Overview](/img/recording/dataarray_indexing_overview.png)


Below we give some examples

In [77]:
# first we pull out a time series to save time in the following
ts = rec["amp"]
display(ts)

Magnitude,[[[0.0913686 0.0909875 0.0910225 ... 0.0941083 0.0940129 0.0944882] [0.1856806 0.186377 0.1836514 ... 0.1856486 0.1850836 0.1842172]] [[0.227516 0.2297024 0.2261366 ... 0.2264519 0.2271665 0.226713] [0.6354927 0.637668 0.6298023 ... 0.6072068 0.6087293 0.6091066]] [[0.1064704 0.1066212 0.1053444 ... 0.121114 0.1205022 0.1205441] [0.2755033 0.2761615 0.2727006 ... 0.2911952 0.2900544 0.2909847]] ... [[0.2027881 0.1996586 0.2004866 ... 0.2318743 0.2311941 0.2330808] [0.4666358 0.4554404 0.4561614 ... 0.4809749 0.4812827 0.4862896]] [[0.4885007 0.4802285 0.4818338 ... 0.6109142 0.6108118 0.613845] [0.8457658 0.825988 0.8259648 ... 0.975894 0.9756599 0.9826459]] [[0.6304559 0.6284427 0.6287045 ... 0.6810626 0.6809573 0.6818709] [1.2285622 1.2205907 1.2190002 ... 1.2729124 1.2727222 1.2755645]]]
Units,volt


it usually helps to know the array's **coordinates**. these can be viewed via the .coords xarray accessor. Note that multiple coordinate axes can overlap. For instance, across the time dimension we can use "time" in seconds or "samples" in integer values. Across the "channel" dimension we can index via Source-Detector pairs (e.g. "S1D1") or via only the "source" or "detector". The latter will give us all matching elements - e.g. "S1" will give us all channels that contain source S1.

In [78]:
display(ts.coords)

Coordinates:
  * time        (time) float64 186kB 0.0 0.128 0.256 ... 2.974e+03 2.974e+03
    samples     (time) int32 93kB 0 1 2 3 4 5 ... 23234 23235 23236 23237 23238
  * channel     (channel) object 224B 'S1D1' 'S1D2' 'S1D3' ... 'S8D8' 'S8D16'
    source      (channel) object 224B 'S1' 'S1' 'S1' 'S1' ... 'S8' 'S8' 'S8'
    detector    (channel) object 224B 'D1' 'D2' 'D3' 'D9' ... 'D7' 'D8' 'D16'
  * wavelength  (wavelength) float64 16B 760.0 850.0

Knowing the coordinates we can also acess the items / labels on the coordinate axes directly

In [79]:
display(ts.wavelength) # wavelength dimension

display(ts.time) # time dimension

<xarray.DataArray 'wavelength' (wavelength: 2)> Size: 16B
array([760., 850.])
Coordinates:
  * wavelength  (wavelength) float64 16B 760.0 850.0

<xarray.DataArray 'time' (time: 23239)> Size: 186kB
array([0.000000e+00, 1.280000e-01, 2.560000e-01, ..., 2.974208e+03,
       2.974336e+03, 2.974464e+03])
Coordinates:
  * time     (time) float64 186kB 0.0 0.128 0.256 ... 2.974e+03 2.974e+03
    samples  (time) int32 93kB 0 1 2 3 4 5 ... 23234 23235 23236 23237 23238
Attributes:
    units:    second

Now we can also 

#### Direct Bracket Indexing
... works as expected

In [80]:
ts[:,0,:] # first item along wavelength

Magnitude,[[0.0913686 0.0909875 0.0910225 ... 0.0941083 0.0940129 0.0944882] [0.227516 0.2297024 0.2261366 ... 0.2264519 0.2271665 0.226713] [0.1064704 0.1066212 0.1053444 ... 0.121114 0.1205022 0.1205441] ... [0.2027881 0.1996586 0.2004866 ... 0.2318743 0.2311941 0.2330808] [0.4885007 0.4802285 0.4818338 ... 0.6109142 0.6108118 0.613845] [0.6304559 0.6284427 0.6287045 ... 0.6810626 0.6809573 0.6818709]]
Units,volt


In [81]:
ts[:,:,::3000] # every 3000th time point

<xarray.DataArray (channel: 28, wavelength: 2, time: 8)> Size: 4kB
<Quantity([[[0.0913686 0.0907284 0.0909799 0.096484  0.0963254 0.0978755 0.0971495
   0.0973881]
  [0.1856806 0.1819877 0.180359  0.190375  0.1934964 0.197304  0.1892043
   0.1941105]]

 [[0.227516  0.2259981 0.2191145 0.2257933 0.2224808 0.224387  0.2252353
   0.2337061]
  [0.6354927 0.6189633 0.5944316 0.6152465 0.6063239 0.6167434 0.5968902
   0.6289392]]

 [[0.1064704 0.1106003 0.1113647 0.1181299 0.1185306 0.12127   0.122234
   0.1231769]
  [0.2755033 0.2795076 0.273957  0.288187  0.2916494 0.3014054 0.2875604
   0.3011461]]

 [[0.0794062 0.0748053 0.0724615 0.0722422 0.0708733 0.0709404 0.0701675
   0.0708531]
  [0.128716  0.1205163 0.1162186 0.1156905 0.1142392 0.1144522 0.1130079
   0.1140507]]

...

 [[0.187484  0.184221  0.1801173 0.179175  0.1779962 0.1764103 0.1753207
   0.1750773]
  [0.2424386 0.2349307 0.2277864 0.2292719 0.2281034 0.2270149 0.2250216
   0.2258399]]

 [[0.2027881 0.2103115 0.2106273 0.2227526 0.2268364 0.227802  0.2291271
   0.2376249]
  [0.4666358 0.4657702 0.4522212 0.4831012 0.4902605 0.5041136 0.4770009
   0.5136656]]

 [[0.4885007 0.5211291 0.532794  0.5610836 0.5836937 0.5948393 0.5834406
   0.5656273]
  [0.8457658 0.876043  0.8738404 0.9376701 0.9678892 0.9856668 0.9351691
   0.8747212]]

 [[0.6304559 0.6529195 0.6589913 0.6675115 0.6719286 0.6761335 0.6776644
   0.6881298]
  [1.2285622 1.2409274 1.2390135 1.2576277 1.264102  1.2721376 1.269668
   1.2924444]]], 'volt')>
Coordinates:
  * time        (time) float64 64B 0.0 384.0 768.0 ... 2.304e+03 2.688e+03
    samples     (time) int32 32B 0 3000 6000 9000 12000 15000 18000 21000
  * channel     (channel) object 224B 'S1D1' 'S1D2' 'S1D3' ... 'S8D8' 'S8D16'
    source      (channel) object 224B 'S1' 'S1' 'S1' 'S1' ... 'S8' 'S8' 'S8'
    detector    (channel) object 224B 'D1' 'D2' 'D3' 'D9' ... 'D7' 'D8' 'D16'
  * wavelength  (wavelength) float64 16B 760.0 850.0
Attributes:
    data_type_group:  unprocessed raw

#### Indexing by Label: .loc and .sel accessors
without using the coordinate we require knowledge of the order of dimensions in the DataArray...

In [82]:
ts.loc["S1D1", 760, :] # time series for channel S1D1 and wavelength 760nm

Magnitude,[0.0913686 0.0909875 0.0910225 ... 0.0941083 0.0940129 0.0944882]
Units,volt


... or we are more explicit, in which case the order does not matter. `.sel` relies on an index. For some  coordinates (time, channel, wavelength) indexes are built. They are printed in bold face when the DataArray is displayed. Indexes are needed for efficient lookup but are not strictly necessary. Hence, we don't always build them by default.

In [83]:
ts.sel(channel="S1D1", wavelength=760)  # the same time series as above

Magnitude,[0.0913686 0.0909875 0.0910225 ... 0.0941083 0.0940129 0.0944882]
Units,volt


`.sel` accepts dictionaries. Useful when dimension name is a variable

In [84]:
dim = 'wavelength'
dim_value = 760
ts.sel({dim : dim_value})

Magnitude,[[0.0913686 0.0909875 0.0910225 ... 0.0941083 0.0940129 0.0944882] [0.227516 0.2297024 0.2261366 ... 0.2264519 0.2271665 0.226713] [0.1064704 0.1066212 0.1053444 ... 0.121114 0.1205022 0.1205441] ... [0.2027881 0.1996586 0.2004866 ... 0.2318743 0.2311941 0.2330808] [0.4885007 0.4802285 0.4818338 ... 0.6109142 0.6108118 0.613845] [0.6304559 0.6284427 0.6287045 ... 0.6810626 0.6809573 0.6818709]]
Units,volt


#### Indexing using logical operations
We can, for instance, choose only those data points that come after t=10s and before t=60s:

In [85]:
ts.sel(time= (ts.time  > 10 ) & (ts.time < 60.))

Magnitude,[[[0.0899084 0.0894057 0.0901775 ... 0.0903297 0.090175 0.0899268] [0.1815226 0.1818538 0.1827682 ... 0.184885 0.1850145 0.1846866]] [[0.2271194 0.2265271 0.2281751 ... 0.2241381 0.2243378 0.2256896] [0.6258998 0.6286326 0.6299954 ... 0.6222512 0.6236847 0.627199]] [[0.106186 0.1064838 0.1053833 ... 0.106494 0.1061651 0.1056465] [0.2714331 0.2728349 0.2736203 ... 0.2736698 0.2742193 0.2759685]] ... [[0.2022316 0.2017929 0.2027171 ... 0.1998214 0.2007051 0.202647] [0.4570127 0.4581691 0.4631743 ... 0.448159 0.4511331 0.4540838]] [[0.487628 0.4916749 0.4934949 ... 0.5004812 0.5035935 0.5044765] [0.8359521 0.8418671 0.8470044 ... 0.8385809 0.8441086 0.8474942]] [[0.6297619 0.6308818 0.6312802 ... 0.637311 0.6379823 0.6392407] [1.2148105 1.2171712 1.2189573 ... 1.2232131 1.2258861 1.2278113]]]
Units,volt


boolean masking works also with the .loc accessor

In [86]:
ts.loc[ts.source == "S1"]

Magnitude,[[[0.0913686 0.0909875 0.0910225 ... 0.0941083 0.0940129 0.0944882] [0.1856806 0.186377 0.1836514 ... 0.1856486 0.1850836 0.1842172]] [[0.227516 0.2297024 0.2261366 ... 0.2264519 0.2271665 0.226713] [0.6354927 0.637668 0.6298023 ... 0.6072068 0.6087293 0.6091066]] [[0.1064704 0.1066212 0.1053444 ... 0.121114 0.1205022 0.1205441] [0.2755033 0.2761615 0.2727006 ... 0.2911952 0.2900544 0.2909847]] [[0.0794062 0.0794713 0.0791685 ... 0.0705146 0.0705608 0.0704029] [0.128716 0.1289733 0.1283794 ... 0.1129772 0.1127544 0.1127884]]]
Units,volt


#### Indexing using stringng matching or "isin"
first via string accessor

In [87]:
# regular expression via str accessor
ts.sel(channel=ts.channel.str.match("S[2,3]D[1,2]"))

Magnitude,[[[0.5512474 0.5510672 0.5476283 ... 0.6179242 0.6188702 0.6187721] [1.125532 1.1238331 1.1119423 ... 1.1817728 1.1819598 1.1832658]] [[0.1765604 0.1768194 0.1764903 ... 0.1787845 0.178888 0.1790632] [0.3228078 0.322831 0.3219485 ... 0.3182213 0.3182161 0.3181564]] [[0.0577035 0.0581161 0.0575968 ... 0.0606553 0.0603812 0.0608511] [0.1558412 0.1549238 0.1535897 ... 0.1503445 0.1506066 0.1506666]] [[0.1679255 0.1679034 0.1677694 ... 0.1610538 0.1611879 0.1610764] [0.3205449 0.3203354 0.3196127 ... 0.2977933 0.2975577 0.2978071]]]
Units,volt


or via the use of `isin` to select a fixed tiem or list of items

In [88]:
# item
ts.sel(channel="S1D1")

# list of items
ts.sel(channel=ts.channel.isin(["S1D1", "S8D8"]))

Magnitude,[[[0.0913686 0.0909875 0.0910225 ... 0.0941083 0.0940129 0.0944882] [0.1856806 0.186377 0.1836514 ... 0.1856486 0.1850836 0.1842172]] [[0.4885007 0.4802285 0.4818338 ... 0.6109142 0.6108118 0.613845] [0.8457658 0.825988 0.8259648 ... 0.975894 0.9756599 0.9826459]]]
Units,volt


#### Building indices if they are not available
Repeat: `.sel` relies on an index. For some  coordinates (time, channel, wavelength) indexes are built. They are printed in bold face when the DataArray is displayed. Indexes are needed for efficient lookup but are not strictly necessary. if we would like to index via a coordinate axis for which no index is available (here the "source" coordinate), they can [be built](https://docs.xarray.dev/en/v2024.07.0/generated/xarray.DataArray.set_xindex.html#xarray.DataArray.set_xindex):

In [89]:
# build the index
ts_with_index = ts.set_xindex("source")
# now we can select by source index
ts_with_index.sel(source="S1")

Magnitude,[[[0.0913686 0.0909875 0.0910225 ... 0.0941083 0.0940129 0.0944882] [0.1856806 0.186377 0.1836514 ... 0.1856486 0.1850836 0.1842172]] [[0.227516 0.2297024 0.2261366 ... 0.2264519 0.2271665 0.226713] [0.6354927 0.637668 0.6298023 ... 0.6072068 0.6087293 0.6091066]] [[0.1064704 0.1066212 0.1053444 ... 0.121114 0.1205022 0.1205441] [0.2755033 0.2761615 0.2727006 ... 0.2911952 0.2900544 0.2909847]] [[0.0794062 0.0794713 0.0791685 ... 0.0705146 0.0705608 0.0704029] [0.128716 0.1289733 0.1283794 ... 0.1129772 0.1127544 0.1127884]]]
Units,volt


#### Using coordinates from one array to index another
Here we use `ts.source` to select in `geo3d` values along the 'label' dimension. Because `ts.source` belongs to the 'channel' dimension of `ts`, the resulting `xr.DataArray` has dimensions 'channel' (from ts.source) and 'digitized' (from geo3d)

In [91]:
display(rec.geo3d)
display(ts.source)
rec.geo3d.loc[ts.source]

Magnitude,[[-0.041613204679326624 0.026799775287857947 0.1299043936308115] [-0.06476686499872276 0.05814256998996063 0.0908425773727145] [-0.07120554551675068 -0.012874272652217859 0.10787860947691345] [-0.0859043654400404 0.018971698468891116 0.06509762433137256] [0.03694171596700852 0.02748380530252158 0.13022129709104263] [0.06065133742692848 0.05882414589197514 0.09117717995727878] [0.06712771392323756 -0.012199231886346213 0.1085725493643022] [0.08188685574250908 0.020427932162352107 0.06571325110115192] [-0.037619588707178915 0.06322851630256272 0.11572802770110814] [-0.04134445059646741 -0.011779611291995052 0.13495002938154654] [-0.07242424650162711 0.02347293206381116 0.10322218957482163] [-0.07912592748234686 0.05140929117919257 0.057370046083468226] [0.03352717285472944 0.06359968341212022 0.11583881331702946] [0.03686639505686032 -0.011397164907962862 0.13536724076864515] [0.06791592703520163 0.02468254467119271 0.10366605207860985] [0.075310088095807 0.05226884499005337 0.05787698428594235] [-0.03773895423262196 0.034082658086024245 0.1294919790818403] [-0.061454307897075164 0.06443800208211416 0.09061004226260877] [-0.07282878975853647 -0.00527870527992114 0.10743054838539287] [-0.08439610638498087 0.02706123378098264 0.06559510739262155] [0.040013338219712126 0.020439745814301982 0.13063767506528579] [0.06428020193514211 0.05162125732852231 0.09133632943784001] [0.06521393141744246 -0.019260368037897515 0.10880928230870081] [0.08272091030272573 0.012990608473329186 0.06658402323335233] [-0.0824899918305801 3.5272652784690273e-09 -8.985265795291575e-10] [6.534060185275914e-12 0.11404663614484922 -8.956669156345853e-09] [0.08248999697928468 3.893090638057428e-09 4.766247813092761e-10] [-0.04018770669918394 0.044642295725887106 0.12357659157001165] [-0.04174110787598461 0.007685839199884737 0.13437743644514044] [-0.05885642692737942 0.026136335712672674 0.11745327806321545] [0.03851939726517181 0.03078283979366837 0.1281798987708399] [-0.052808259274512416 0.06188780045911764 0.10403189889709587] [-0.06922421433143165 0.04108974212744533 0.0972095427514579] [-0.07351067792317667 0.05556043944468993 0.07592438053279707] [0.031418413207589764 0.05609701242391968 0.12112072355182572] [-0.05798909959739463 -0.013176608236818512 0.12293491786782985] [-0.07419549041066105 0.00549319711704184 0.10669158774707252] [0.023828589288117673 0.003996293896988853 0.14081960191362605] [-0.08184195746940305 0.022098763709400317 0.08422485645365857] [-0.08309090979477994 0.03520830182778492 0.0610323268526085] [0.026425335781627597 0.043375220063173334 0.12939415639453739] [0.035781770725957916 0.04568049698512951 0.12354633505671649] [0.037556277679295876 0.008001190852073497 0.134555406719549] [0.054080633753515184 0.026705026363902817 0.11818464121484251] [0.06801076312164317 0.02284818084680601 0.10445872040876931] [0.0481712114118228 0.062034828924799924 0.10518292837437133] [0.06461419274656584 0.042329086589563227 0.0977770136200937] [0.06967032405092426 0.05604290375898291 0.07590918892077606] [0.07526945345643418 0.037578919664336574 0.08007431341875021] [0.05288202968424921 -0.012139614237985343 0.12430811820847815] [0.06938013615060833 0.0069016618704050535 0.10806432085401242] [0.07932239267332403 -0.00044800545870984573 0.08903670686337778] [0.07756024143390347 0.022515587935674892 0.08578432016555051] [0.07912303758068968 0.036496511669642975 0.06257633565753129] [0.08225821050778642 0.01751423306824721 0.06619028345490524]]
Units,meter


<xarray.DataArray 'source' (channel: 28)> Size: 224B
array(['S1', 'S1', 'S1', 'S1', 'S2', 'S2', 'S2', 'S2', 'S3', 'S3', 'S3',
       'S4', 'S4', 'S4', 'S5', 'S5', 'S5', 'S5', 'S6', 'S6', 'S6', 'S6',
       'S7', 'S7', 'S7', 'S8', 'S8', 'S8'], dtype=object)
Coordinates:
  * channel   (channel) object 224B 'S1D1' 'S1D2' 'S1D3' ... 'S8D8' 'S8D16'
    source    (channel) object 224B 'S1' 'S1' 'S1' 'S1' ... 'S7' 'S8' 'S8' 'S8'
    detector  (channel) object 224B 'D1' 'D2' 'D3' 'D9' ... 'D7' 'D8' 'D16'

Magnitude,[[-0.041613204679326624 0.026799775287857947 0.1299043936308115] [-0.041613204679326624 0.026799775287857947 0.1299043936308115] [-0.041613204679326624 0.026799775287857947 0.1299043936308115] [-0.041613204679326624 0.026799775287857947 0.1299043936308115] [-0.06476686499872276 0.05814256998996063 0.0908425773727145] [-0.06476686499872276 0.05814256998996063 0.0908425773727145] [-0.06476686499872276 0.05814256998996063 0.0908425773727145] [-0.06476686499872276 0.05814256998996063 0.0908425773727145] [-0.07120554551675068 -0.012874272652217859 0.10787860947691345] [-0.07120554551675068 -0.012874272652217859 0.10787860947691345] [-0.07120554551675068 -0.012874272652217859 0.10787860947691345] [-0.0859043654400404 0.018971698468891116 0.06509762433137256] [-0.0859043654400404 0.018971698468891116 0.06509762433137256] [-0.0859043654400404 0.018971698468891116 0.06509762433137256] [0.03694171596700852 0.02748380530252158 0.13022129709104263] [0.03694171596700852 0.02748380530252158 0.13022129709104263] [0.03694171596700852 0.02748380530252158 0.13022129709104263] [0.03694171596700852 0.02748380530252158 0.13022129709104263] [0.06065133742692848 0.05882414589197514 0.09117717995727878] [0.06065133742692848 0.05882414589197514 0.09117717995727878] [0.06065133742692848 0.05882414589197514 0.09117717995727878] [0.06065133742692848 0.05882414589197514 0.09117717995727878] [0.06712771392323756 -0.012199231886346213 0.1085725493643022] [0.06712771392323756 -0.012199231886346213 0.1085725493643022] [0.06712771392323756 -0.012199231886346213 0.1085725493643022] [0.08188685574250908 0.020427932162352107 0.06571325110115192] [0.08188685574250908 0.020427932162352107 0.06571325110115192] [0.08188685574250908 0.020427932162352107 0.06571325110115192]]
Units,meter


### Accessing xarray DataArray values with .values
e.g. to write them to a numpy array. Example: We want to pull out the actual source names of the first 3 sources..

In [92]:
# this way of indexing will not give us what we want, as it returns another xarray with coordinates etc.
display(ts.source[:3])

# instead we use the .values accessors:
display(ts.source.values[:3])

<xarray.DataArray 'source' (channel: 3)> Size: 24B
array(['S1', 'S1', 'S1'], dtype=object)
Coordinates:
  * channel   (channel) object 24B 'S1D1' 'S1D2' 'S1D3'
    source    (channel) object 24B 'S1' 'S1' 'S1'
    detector  (channel) object 24B 'D1' 'D2' 'D3'

array(['S1', 'S1', 'S1'], dtype=object)

### Accessing single items in an xarray with .item
indexing a single item in an xarray is still an xarray with coordinates

In [93]:
display(ts[0,0,0]) # the first time point of the first channel and first wavelength in the DataArray

<xarray.DataArray ()> Size: 8B
<Quantity(0.0913686, 'volt')>
Coordinates:
    time        float64 8B 0.0
    samples     int32 4B 0
    channel     <U4 16B 'S1D1'
    source      object 8B 'S1'
    detector    object 8B 'D1'
    wavelength  float64 8B 760.0
Attributes:
    data_type_group:  unprocessed raw

In [94]:
# to get just the item we use .item()
display(ts[0,0,0].item())

<Quantity(0.0913686, 'volt')>

In [95]:
# of course this also works with the .sel method
ts.sel(channel="S1D1", wavelength= "760", time = "0.0").item()

<Quantity(0.0913686, 'volt')>